## Topic Modeling 

In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt


In [11]:
filename = "sample-S2-records"
df = pd.read_json(filename, lines=True) 
abstract = df['paperAbstract']
abstract.replace('', np.NaN)


absDictionary = abstract.to_dict()
absData = []
for key in absDictionary:
    if absDictionary[key] =='':
        continue
    else:
        absData.append(absDictionary[key])

print(len(absData))

58


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

no_features = 1000

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(absData)
tf_feature_names = tf_vectorizer.get_feature_names()

In [13]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

no_topics = 20
# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

/Users/nahalam/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


## Evaluating Topics

In [14]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 10
display_topics(lda, tf_feature_names, no_top_words)

Topic 0:
19 ca ip patients positive level sera mean ml 05
Topic 1:
time children hypothermia days immunized hydrogen later conditions medical work
Topic 2:
health stroke self poor disability associated based ci 95 factors
Topic 3:
control optimal network neural method fed performance ii paper flow
Topic 4:
skin technology sensor using influence role common epidermal dna studied
Topic 5:
add rna paper formation enzyme dna children shown using maximum
Topic 6:
fusion stages function study observations completely suggest differences related days
Topic 7:
mg simultaneously state predict years model rate 50 achieved applying
Topic 8:
authors body min organs 30 al currently series visceral et
Topic 9:
liver symptoms patients clinical metastasis signs findings compromise metastases synchronous
Topic 10:
essential local days mainly dependent hospital device hypothermia future activity
Topic 11:
living related peptide cancer activity knowledge levels fed care based
Topic 12:
tumor cells oxygen 